## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint as pp

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
0,0,Nicoya,Cr,10.1483,-85.4520,84.13,67,98,5.91,overcast clouds
1,1,Vaini,To,-21.2000,-175.2000,75.36,78,40,6.91,scattered clouds
2,2,Antalaha,Mg,-14.9003,50.2788,70.18,81,52,7.63,broken clouds
3,3,Kapaa,Us,22.0752,-159.3190,81.12,86,40,12.66,scattered clouds
4,4,Juneau,Us,58.3019,-134.4197,63.84,66,75,8.05,heavy intensity rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
0,0,Nicoya,Cr,10.1483,-85.4520,84.13,67,98,5.91,overcast clouds
1,1,Vaini,To,-21.2000,-175.2000,75.36,78,40,6.91,scattered clouds
3,3,Kapaa,Us,22.0752,-159.3190,81.12,86,40,12.66,scattered clouds
6,6,Port Blair,In,11.6667,92.7500,75.24,88,100,11.50,thunderstorm with light rain
24,24,Isla,Ph,15.8939,120.3036,76.17,93,100,3.69,overcast clouds
29,29,San Andres,Co,12.5847,-81.7006,86.02,74,75,8.05,broken clouds
35,35,Rikitea,Pf,-23.1203,-134.9692,75.72,80,90,11.68,light rain
41,41,San Cristobal,Ve,7.7669,-72.2250,80.78,61,100,7.43,overcast clouds
44,44,Avarua,Ck,-21.2078,-159.7750,75.25,94,75,17.69,broken clouds
46,46,Butaritari,Ki,3.0707,172.7902,81.45,69,100,13.94,overcast clouds


In [39]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Weather       0
dtype: int64

In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID       0
City          0
Country       0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Weather       0
dtype: int64

In [42]:
clean_df.count()

City_ID       217
City          217
Country       217
Lat           217
Lng           217
Max Temp      217
Humidity      217
Cloudiness    217
Wind Speed    217
Weather       217
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
0,Nicoya,Cr,84.13,overcast clouds,10.1483,-85.4520,
1,Vaini,To,75.36,scattered clouds,-21.2000,-175.2000,
3,Kapaa,Us,81.12,scattered clouds,22.0752,-159.3190,
6,Port Blair,In,75.24,thunderstorm with light rain,11.6667,92.7500,
24,Isla,Ph,76.17,overcast clouds,15.8939,120.3036,
29,San Andres,Co,86.02,broken clouds,12.5847,-81.7006,
35,Rikitea,Pf,75.72,light rain,-23.1203,-134.9692,
41,San Cristobal,Ve,80.78,overcast clouds,7.7669,-72.2250,
44,Avarua,Ck,75.25,broken clouds,-21.2078,-159.7750,
46,Butaritari,Ki,81.45,overcast clouds,3.0707,172.7902,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6b. Iterate through the hotel DataFrame.
for i in hotel_df.index:

    # 6c. Get latitude and longitude from DataFrame
    city = hotel_df.loc[i, "City"]
    country = hotel_df.loc[i, "Country"]
    lat = hotel_df.loc[i, "Lat"]
    lng = hotel_df.loc[i, "Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for Index {i}: {city}, {country}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        name = results[0]['name']
        hotel_df.loc[i, 'Hotel Name'] = name
        print(f"The nearest hotel in/to {city}, {country} is {name}.")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

hotel_df.head()

Retrieving Results for Index 0: Nicoya, Cr.
The nearest hotel in/to Nicoya, Cr is Las Cabinas Del Sueño.
------------
Retrieving Results for Index 1: Vaini, To.
The nearest hotel in/to Vaini, To is Keleti Beach Resort.
------------
Retrieving Results for Index 3: Kapaa, Us.
The nearest hotel in/to Kapaa, Us is Sheraton Kauai Coconut Beach Resort.
------------
Retrieving Results for Index 6: Port Blair, In.
The nearest hotel in/to Port Blair, In is Welcomhotel By ITC Hotels, Bay Island, Port Blair.
------------
Retrieving Results for Index 24: Isla, Ph.
The nearest hotel in/to Isla, Ph is Green Apple Inn.
------------
Retrieving Results for Index 29: San Andres, Co.
The nearest hotel in/to San Andres, Co is Hotel Arena Blanca.
------------
Retrieving Results for Index 35: Rikitea, Pf.
The nearest hotel in/to Rikitea, Pf is People ThankYou.
------------
Retrieving Results for Index 41: San Cristobal, Ve.
The nearest hotel in/to San Cristobal, Ve is Pirineos.
------------
Retrieving Resul

The nearest hotel in/to Camopi, Gf is La pousada.
------------
Retrieving Results for Index 243: Manokwari, Id.
The nearest hotel in/to Manokwari, Id is Swiss Belhotel Manokwari.
------------
Retrieving Results for Index 249: Bourg-En-Bresse, Fr.
The nearest hotel in/to Bourg-En-Bresse, Fr is Chambres d'Hôtes Grange Carrée.
------------
Retrieving Results for Index 251: Sibu, My.
The nearest hotel in/to Sibu, My is RH HOTEL (4 Star).
------------
Retrieving Results for Index 252: Mogadishu, So.
The nearest hotel in/to Mogadishu, So is Hotel Juba.
------------
Retrieving Results for Index 253: La Libertad, Ec.
The nearest hotel in/to La Libertad, Ec is Playa Canela Hotel Boutique.
------------
Retrieving Results for Index 254: Jibuti, Dj.
The nearest hotel in/to Jibuti, Dj is Orchid Main Camp Goubet.
------------
Retrieving Results for Index 255: Osmena, Ph.
The nearest hotel in/to Osmena, Ph is Arjemarkyza Lodge.
------------
Retrieving Results for Index 257: Rio Branco, Br.
The neares

The nearest hotel in/to Bambanglipuro, Id is Puri Brata.
------------
Retrieving Results for Index 423: Hambantota, Lk.
The nearest hotel in/to Hambantota, Lk is Bungalow 63.
------------
Retrieving Results for Index 430: Shinpokh, Pk.
Missing field/result... skipping.
------------
Retrieving Results for Index 431: Auki, Sb.
The nearest hotel in/to Auki, Sb is Auki Motel.
------------
Retrieving Results for Index 432: Wattegama, Lk.
Missing field/result... skipping.
------------
Retrieving Results for Index 433: Cayenne, Gf.
The nearest hotel in/to Cayenne, Gf is Hôtel Le Dronmi.
------------
Retrieving Results for Index 440: Thinadhoo, Mv.
Missing field/result... skipping.
------------
Retrieving Results for Index 443: Birao, Cf.
Missing field/result... skipping.
------------
Retrieving Results for Index 445: Jalu, Ly.
The nearest hotel in/to Jalu, Ly is صاحبة الفخامة.
------------
Retrieving Results for Index 447: Ruidoso, Us.
The nearest hotel in/to Ruidoso, Us is Inn of the Mountai

The nearest hotel in/to Kattivakkam, In is Euronova Freezer.
------------
Retrieving Results for Index 624: Miri, My.
The nearest hotel in/to Miri, My is Mega Hotel, Miri.
------------
Retrieving Results for Index 625: Nieuw Amsterdam, Sr.
The nearest hotel in/to Nieuw Amsterdam, Sr is Hotel Stardust.
------------
Retrieving Results for Index 627: Mae Sot, Th.
The nearest hotel in/to Mae Sot, Th is Suan Darb Porn Resort (บ้านสวนดาบพรรีสอร์ท).
------------
Retrieving Results for Index 633: Sao Jose Da Coroa Grande, Br.
The nearest hotel in/to Sao Jose Da Coroa Grande, Br is Crôa Mares Hotel.
------------
Retrieving Results for Index 636: Fasa, Ir.
The nearest hotel in/to Fasa, Ir is Fasa University Boys' Dormitory.
------------
Retrieving Results for Index 638: Praia, Cv.
The nearest hotel in/to Praia, Cv is LT Aparthotel.
------------
Retrieving Results for Index 651: Grand-Lahou, Ci.
The nearest hotel in/to Grand-Lahou, Ci is Hotel Beatitude de Grand Lahou.
------------
Retrieving Res

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
0,Nicoya,Cr,84.13,overcast clouds,10.1483,-85.4520,Las Cabinas Del Sueño
1,Vaini,To,75.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
3,Kapaa,Us,81.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Port Blair,In,75.24,thunderstorm with light rain,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
24,Isla,Ph,76.17,overcast clouds,15.8939,120.3036,Green Apple Inn


In [41]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()

City          200
Country       200
Max Temp      200
Weather       200
Lat           200
Lng           200
Hotel Name    200
dtype: int64

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [45]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))